In [17]:
import numpy as np
import pandas as pd
from pathlib import Path
from bs4 import BeautifulSoup
import datetime
import glob
import os

files = glob.glob('./*.html')

In [13]:
files

['./awrrpt_1_209_210.html',
 './awrrpt_rac_209_210.html',
 './awrrpt_2_209_210.html']

In [14]:
table_list = {
    'inst_info':  'This table displays database instance information',
    'host_info':  'This table displays host information',
    'snap_info':  'This table displays snapshot information',
    'inst_eff':   'This table displays instance efficiency percentages',
    'wait_event': 'This table displays top 10 wait events by total wait time'
}

In [15]:
for k, v in table_list.items():
    print(v)

This table displays database instance information
This table displays host information
This table displays snapshot information
This table displays instance efficiency percentages
This table displays top 10 wait events by total wait time


In [16]:
df_inst_eff = pd.DataFrame()
df_wait = pd.DataFrame()

for f in files:
    soup = BeautifulSoup(open(f), 'html.parser')
    
    for k, v in table_list.items():
        table = soup.find_all('table', {'summary' : v})
        
        if k == 'inst_info':
            dfs = pd.read_html(str(table),header=0)
            db_name = dfs[0]['DB Name'][0]
            inst_name = dfs[1]['Instance'][0]
            
        if k == 'host_info':
            dfs = pd.read_html(str(table),header=0)
            cores = dfs[0]['Cores'][0]
            mem_GB = dfs[0]['Memory (GB)'][0] 

        if k == 'snap_info':
            dfs = pd.read_html(str(table),header=0)
            begin_snap = dfs[0]['Snap Id'][0].astype(int)
            begin_snap_time = datetime.datetime.strptime(dfs[0]['Snap Time'][0], '%d-%b-%y %H:%M:%S').strftime('%Y/%m/%d %H:%M:%S')
            
            end_snap = dfs[0]['Snap Id'][1].astype(int)
            end_snap_time = datetime.datetime.strptime(dfs[0]['Snap Time'][1], '%d-%b-%y %H:%M:%S').strftime('%Y/%m/%d %H:%M:%S')
        
        if k == 'inst_eff':
            dfs = pd.read_html(str(table),header=None)
            tmp = dfs[0]

            tmp1 = tmp[[0, 1]]
            tmp2 = tmp[[2, 3]]

            tmp1 = tmp1.rename(columns={0: 'name', 1: 'pct'})
            tmp2 = tmp2.rename(columns={2: 'name', 3: 'pct'})

            tmp = pd.concat([tmp1, tmp2]).dropna()
            
            tmp.insert(0, 'end_snap_time', end_snap_time)
            tmp.insert(0, 'begin_snap_time', begin_snap_time)
            tmp.insert(0, 'end_snap', end_snap)
            tmp.insert(0, 'begin_snap', begin_snap)
            tmp.insert(0, 'inst_name', inst_name)
            tmp.insert(0, 'db_name', db_name)
            
            df_inst_eff = pd.concat([df_inst_eff, tmp])
            
        if k == 'wait_event':
            dfs = pd.read_html(str(table),header=0)
            tmp = dfs[0]
            
            tmp.insert(0, 'end_snap_time', end_snap_time)
            tmp.insert(0, 'begin_snap_time', begin_snap_time)
            tmp.insert(0, 'end_snap', end_snap)
            tmp.insert(0, 'begin_snap', begin_snap)
            tmp.insert(0, 'inst_name', inst_name)
            tmp.insert(0, 'db_name', db_name)
            
            df_wait = pd.concat([df_wait, tmp])
  

ImportError: lxml not found, please install it

In [349]:
print("db_name: {}".format(db_name))
print("inst_name: {}".format(inst_name))
print("core: {}".format(cores))
print("mem_GB: {}".format(mem_GB))
print("begin_snap:\t{}".format(begin_snap), "{}".format(begin_snap_time))
print("end_snap:\t{}".format(end_snap), "{}".format(end_snap_time))

df_inst_eff

db_name: ORCL
inst_name: orcl2
core: 2
mem_GB: 4.58
begin_snap:	209 2020/05/31 03:50:41
end_snap:	210 2020/05/31 04:04:22


,db_name,inst_name,begin_snap,end_snap,begin_snap_time,end_snap_time,name,pct
0,ORCL,orcl1,209,210,2020/05/31 03:50:41,2020/05/31 04:04:22,Buffer Nowait %:,99.99
1,ORCL,orcl1,209,210,2020/05/31 03:50:41,2020/05/31 04:04:22,Buffer Hit %:,99.30
2,ORCL,orcl1,209,210,2020/05/31 03:50:41,2020/05/31 04:04:22,Library Hit %:,86.62
3,ORCL,orcl1,209,210,2020/05/31 03:50:41,2020/05/31 04:04:22,Execute to Parse %:,73.60
4,ORCL,orcl1,209,210,2020/05/31 03:50:41,2020/05/31 04:04:22,Parse CPU to Parse Elapsd %:,68.65
5,ORCL,orcl1,209,210,2020/05/31 03:50:41,2020/05/31 04:04:22,Flash Cache Hit %:,0.00
0,ORCL,orcl1,209,210,2020/05/31 03:50:41,2020/05/31 04:04:22,Redo NoWait %:,100.00
1,ORCL,orcl1,209,210,2020/05/31 03:50:41,2020/05/31 04:04:22,In-memory Sort %:,100.00
2,ORCL,orcl1,209,210,2020/05/31 03:50:41,2020/05/31 04:04:22,Soft Parse %:,87.97
3,ORCL,orcl1,209,210,2020/05/31 03:50:41,2020/05/31 04:04:22,Latch Hit %:,99.98


In [358]:
# pd.pivot_table(df_inst_eff, index=['begin_snap_time','end_snap_time','name'], columns='inst_name', values='pct')
pd.pivot_table(df_inst_eff, index=['begin_snap_time','end_snap_time','inst_name'], columns='name', values='pct')

name                                               % Non-Parse CPU:  \
begin_snap_time     end_snap_time       inst_name                     
2020/05/31 03:50:41 2020/05/31 04:04:22 orcl1                 96.91   
                                        orcl2                 97.37   

name                                               Buffer Hit %:  \
begin_snap_time     end_snap_time       inst_name                  
2020/05/31 03:50:41 2020/05/31 04:04:22 orcl1              99.30   
                                        orcl2              96.11   

name                                               Buffer Nowait %:  \
begin_snap_time     end_snap_time       inst_name                     
2020/05/31 03:50:41 2020/05/31 04:04:22 orcl1                 99.99   
                                        orcl2                 99.99   

name                                               Execute to Parse %:  \
begin_snap_time     end_snap_time       inst_name                        
2020/05/31 03:50:41 2020/05/31 04:04:22 orcl1                    73.60   
                                        orcl2                    74.28   

name                                               Flash Cache Hit %:  \
begin_snap_time     end_snap_time       inst_name                       
2020/05/31 03:50:41 2020/05/31 04:04:22 orcl1                     0.0   
                                        orcl2                     0.0   

name                                               In-memory Sort %:  \
begin_snap_time     end_snap_time       inst_name                      
2020/05/31 03:50:41 2020/05/31 04:04:22 orcl1                  100.0   
                                        orcl2                  100.0   

name                                               Latch Hit %:  \
begin_snap_time     end_snap_time       inst_name                 
2020/05/31 03:50:41 2020/05/31 04:04:22 orcl1             99.98   
                                        orcl2             99.97   

name                                               Library Hit %:  \
begin_snap_time     end_snap_time       inst_name                   
2020/05/31 03:50:41 2020/05/31 04:04:22 orcl1               86.62   
                                        orcl2               86.67   

name                                               Parse CPU to Parse Elapsd %:  \
begin_snap_time     end_snap_time       inst_name                                 
2020/05/31 03:50:41 2020/05/31 04:04:22 orcl1                             68.65   
                                        orcl2                             67.52   

name                                               Redo NoWait %:  \
begin_snap_time     end_snap_time       inst_name                   
2020/05/31 03:50:41 2020/05/31 04:04:22 orcl1               100.0   
                                        orcl2               100.0   

name                                               Soft Parse %:  
begin_snap_time     end_snap_time       inst_name                 
2020/05/31 03:50:41 2020/05/31 04:04:22 orcl1              87.97  
                                        orcl2              88.20

In [353]:
df_wait.head()

,db_name,inst_name,begin_snap,end_snap,begin_snap_time,end_snap_time,Event,Waits,Total Wait Time (sec),Avg Wait,% DB time,Wait Class
0,ORCL,orcl1,209,210,2020/05/31 03:50:41,2020/05/31 04:04:22,DB CPU,NaN,3.8,NaN,51.6,NaN
1,ORCL,orcl1,209,210,2020/05/31 03:50:41,2020/05/31 04:04:22,gc quiesce,1.0,1.0,1009.92ms,13.8,Cluster
2,ORCL,orcl1,209,210,2020/05/31 03:50:41,2020/05/31 04:04:22,gc current block 2-way,630.0,0.5,802.80us,6.9,Cluster
3,ORCL,orcl1,209,210,2020/05/31 03:50:41,2020/05/31 04:04:22,gc current block busy,280.0,0.3,1.15ms,4.4,Cluster
4,ORCL,orcl1,209,210,2020/05/31 03:50:41,2020/05/31 04:04:22,gc cr multi block mixed,58.0,0.2,2.99ms,2.4,Cluster


In [360]:
# pd.pivot_table(df_wait, index=['begin_snap_time','end_snap_time','Event'], columns='inst_name', values='% DB time')
pd.pivot_table(df_wait, index=['begin_snap_time','end_snap_time','inst_name'], columns='Event', values='% DB time')


Event                                              DB CPU  \
begin_snap_time     end_snap_time       inst_name           
2020/05/31 03:50:41 2020/05/31 04:04:22 orcl1        51.6   
                                        orcl2        81.2   

Event                                              Disk file operations I/O  \
begin_snap_time     end_snap_time       inst_name                             
2020/05/31 03:50:41 2020/05/31 04:04:22 orcl1                           NaN   
                                        orcl2                           0.8   

Event                                              IPC group service call  \
begin_snap_time     end_snap_time       inst_name                           
2020/05/31 03:50:41 2020/05/31 04:04:22 orcl1                         NaN   
                                        orcl2                         0.0   

Event                                              IPC send completion sync  \
begin_snap_time     end_snap_time       inst_name                             
2020/05/31 03:50:41 2020/05/31 04:04:22 orcl1                           NaN   
                                        orcl2                           8.8   

Event                                              PGA memory operation  \
begin_snap_time     end_snap_time       inst_name                         
2020/05/31 03:50:41 2020/05/31 04:04:22 orcl1                       NaN   
                                        orcl2                       0.2   

Event                                              PX Deq: Slave Session Stats  \
begin_snap_time     end_snap_time       inst_name                                
2020/05/31 03:50:41 2020/05/31 04:04:22 orcl1                              NaN   
                                        orcl2                              3.9   

Event                                              PX Deq: reap credit  \
begin_snap_time     end_snap_time       inst_name                        
2020/05/31 03:50:41 2020/05/31 04:04:22 orcl1                      NaN   
                                        orcl2                      0.4   

Event                                              control file sequential read  \
begin_snap_time     end_snap_time       inst_name                                 
2020/05/31 03:50:41 2020/05/31 04:04:22 orcl1                               1.4   
                                        orcl2                              16.8   

Event                                              db file sequential read  \
begin_snap_time     end_snap_time       inst_name                            
2020/05/31 03:50:41 2020/05/31 04:04:22 orcl1                          2.1   
                                        orcl2                          NaN   

Event                                              enq: PS - contention  \
begin_snap_time     end_snap_time       inst_name                         
2020/05/31 03:50:41 2020/05/31 04:04:22 orcl1                       NaN   
                                        orcl2                       4.5   

Event                                              enq: TM - contention  \
begin_snap_time     end_snap_time       inst_name                         
2020/05/31 03:50:41 2020/05/31 04:04:22 orcl1                       1.5   
                                        orcl2                       NaN   

Event                                              gc cr block 2-way  \
begin_snap_time     end_snap_time       inst_name                      
2020/05/31 03:50:41 2020/05/31 04:04:22 orcl1                    1.8   
                                        orcl2                    NaN   

Event                                              gc cr multi block mixed  \
begin_snap_time     end_snap_time       inst_name                            
2020/05/31 03:50:41 2020/05/31 04:04:22 orcl1                          2.4   
                                        orcl2                          NaN   

Event                              